<a href="https://colab.research.google.com/github/estorl03-tech/AI-AGENT/blob/main/news-auto-agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langgraph langchain_openai langchain_core tweepy requests

## 応用AIエージェント

- ニュースを取得
- 文書を要約
- Tweetを投稿する


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.prebuilt import create_react_agent
from google.colab import userdata

import gradio as gr
import requests
import tweepy

## APIキー
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
NEWS_API_KEY = userdata.get('NEWS_API_KEY')
BEARER_TOKEN = userdata.get('BEARER_TOKEN')
X_API_KEY = userdata.get('X_API_KEY')
X_API_KEY_SECRET = userdata.get('X_API_KEY_SECRET')
ACCESS_TOKEN = userdata.get('ACCESS_TOKEN')
ACCESS_TOKEN_SECRET = userdata.get('ACCESS_TOKEN_SECRET')

# 最新のニュースをひとつ取得するツール
@tool
def get_news(query: str) -> str:
  """キーワードに合致する最新ニュースを一件取得してタイトルと説明を返す"""
  url = "http://newsapi.org/v2/everything"
  params = {
      "q": query,
      "sortBy": "publishedAt",
      "pageSize": 1,
      "apiKey": NEWS_API_KEY
  }

  article = requests.get(url, params).json()["articles"][0]
  return f"{article['title']} - {article['description']}"

# 140字以内で要約するツール
@tool
def summarize(text: str) -> str:
  """テキストを要約して返す"""
  llm = ChatOpenAI(model = "gpt-4o-mini-2024-07-18", temperature = 0, openai_api_key = OPENAI_API_KEY)
  return llm.invoke([SystemMessage(content="以下のテキストを140字以内で簡潔に要約してください。）"), HumanMessage(content = text)]).content

# ツイートをするツール
@tool
def post_tweet(text: str) -> str:
  """テキストをツイートする"""
  client = tweepy.Client(
    bearer_token = BEARER_TOKEN,
    consumer_key = X_API_KEY,
    consumer_secret = X_API_KEY_SECRET,
    access_token = ACCESS_TOKEN,
    access_token_secret = ACCESS_TOKEN_SECRET
  )

  tweet = client.create_tweet(text = text)
  return f"{tweet.data['id']}"

# AIエージェント作成
def run_agent(user_input: str) -> str:
  tools = [get_news, summarize, post_tweet]

  system_prompt = "ユーザーの入力に対して、get_newsとsummarizeとpost_tweetから適切なツールを使って答えてください"

  llm = ChatOpenAI(model = "gpt-4o-mini-2024-07-18", temperature = 0, openai_api_key = OPENAI_API_KEY)
  agent = create_react_agent(llm, tools, prompt=system_prompt)

  response =  agent.invoke({"messages": [HumanMessage(content = user_input)]})
  print(response["messages"][-1].content)

  return response

# run_agentをそのまま使う
def agent_interface(user_input):
    try:
        response = run_agent(user_input)
        # LangGraphは messages を返すので、最後のAIMessage.contentを取り出す
        messages = response["messages"]
        last_message = messages[-1]

        # AIMessageの本文
        if hasattr(last_message, "content"):
          output_text = last_message.content
        else:
          output_text = str(last_message)

        return f"✅ 完了しました！\n\n{output_text}"

    except Exception as e:
        return f"エラー: {e}"

with gr.Blocks() as demo:
    gr.Markdown("## ニュース取得 → 要約 → ツイート AIエージェント Demo")

    with gr.Row():
        with gr.Column(scale=3):
            user_input = gr.Textbox(
                label="入力（例: AI, 経済, スポーツ）",
                placeholder="気になるトピックを入力してください"
            )
            run_button = gr.Button("実行")

        with gr.Column(scale=4):
            output = gr.Textbox(
                label="出力",
                lines=5
            )

    run_button.click(fn=agent_interface, inputs=user_input, outputs=output)

demo.launch()

In [ ]:
user_input = "人工知能に関するニュースを要約してツイートして"
response = run_agent(user_input)